In [ ]:
#Configure weights & biases
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.8/257.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.0 MB/s eta 0:00:00


In [ ]:
#instll all pckages
import wandb
import numpy as np
import torch
import torch.nn as nn #base class for neutral networks

import pandas as pd
from sklearn.preprocessing import StandardScaler

from torch.utils.data import Dataset, DataLoader

In [ ]:
# setting up the wandb configurations
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
#define configs
wandb.init(
    project ='Bank Marketing',
    config= {
        "epochs" : 50 ,
        "batch_size": 64 ,
        "lr" : 0.01
    }
)

config = wandb.config


In [ ]:
#importing data
data = pd.read_csv('bank.csv')
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes


In [ ]:
data.tail()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
11157,33,blue-collar,single,primary,no,1,yes,no,cellular,20,apr,257,1,-1,0,unknown,no
11158,39,services,married,secondary,no,733,no,no,unknown,16,jun,83,4,-1,0,unknown,no
11159,32,technician,single,secondary,no,29,no,no,cellular,19,aug,156,2,-1,0,unknown,no
11160,43,technician,married,secondary,no,0,no,yes,cellular,8,may,9,2,172,5,failure,no
11161,34,technician,married,secondary,no,0,no,no,cellular,9,jul,628,1,-1,0,unknown,no


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        11162 non-null  int64 
 1   job        11162 non-null  object
 2   marital    11162 non-null  object
 3   education  11162 non-null  object
 4   default    11162 non-null  object
 5   balance    11162 non-null  int64 
 6   housing    11162 non-null  object
 7   loan       11162 non-null  object
 8   contact    11162 non-null  object
 9   day        11162 non-null  int64 
 10  month      11162 non-null  object
 11  duration   11162 non-null  int64 
 12  campaign   11162 non-null  int64 
 13  pdays      11162 non-null  int64 
 14  previous   11162 non-null  int64 
 15  poutcome   11162 non-null  object
 16  deposit    11162 non-null  object
dtypes: int64(7), object(10)
memory usage: 1.4+ MB


In [ ]:
categorical_columns = data.select_dtypes(include=['object']).columns

# Display the list of categorical columns
print("Categorical Columns:")
print(categorical_columns)

# Extract the categorical columns into a new DataFrame if needed
X_categorical = data[categorical_columns]

Categorical Columns:
Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'poutcome', 'deposit'],
      dtype='object')


#Preprocessing

In [ ]:
categorical_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']

for column in categorical_columns:
    print(data[column].value_counts())

management       2566
blue-collar      1944
technician       1823
admin.           1334
services          923
retired           778
self-employed     405
student           360
unemployed        357
entrepreneur      328
housemaid         274
unknown            70
Name: job, dtype: int64
married     6351
single      3518
divorced    1293
Name: marital, dtype: int64
secondary    5476
tertiary     3689
primary      1500
unknown       497
Name: education, dtype: int64
no     10994
yes      168
Name: default, dtype: int64
no     5881
yes    5281
Name: housing, dtype: int64
no     9702
yes    1460
Name: loan, dtype: int64
cellular     8042
unknown      2346
telephone     774
Name: contact, dtype: int64
may    2824
aug    1519
jul    1514
jun    1222
nov     943
apr     923
feb     776
oct     392
jan     344
sep     319
mar     276
dec     110
Name: month, dtype: int64
unknown    8326
failure    1228
success    1071
other       537
Name: poutcome, dtype: int64


In [ ]:
# Define the mapping dictionary
mapping = {
    'management': 0,
    'blue-collar': 1,
    'technician': 2,
    'admin.': 3,
    'services': 4,
    'retired': 5,
    'self-employed': 6,
    'student': 7,
    'unemployed': 8,
    'entrepreneur': 9,
    'housemaid': 10,
    'unknown': 11
}

# Replace values in the "job" column using the mapping dictionary
data['job'] = data['job'].map(mapping)

# Display the result
print(data[['job']])

       job
0        3
1        3
2        2
3        4
4        3
...    ...
11157    1
11158    4
11159    2
11160    2
11161    2

[11162 rows x 1 columns]


In [ ]:
# Define the mapping dictionary
mapping_marital = {
    'married': 0,
    'single': 1,
    'divorced': 2
}

# Replace values in the "marital" column using the mapping dictionary
data['marital'] = data['marital'].map(mapping_marital)

# Display the result
print(data[['marital']])

       marital
0            0
1            0
2            0
3            0
4            0
...        ...
11157        1
11158        0
11159        1
11160        0
11161        0

[11162 rows x 1 columns]


In [ ]:
# Define the mapping dictionary
mapping_education = {
    'secondary': 0,
    'tertiary': 1,
    'primary': 2,
    'unknown': 3
}

# Replace values in the "education" column using the mapping dictionary
data['education'] = data['education'].map(mapping_education)

# Display the result
print(data[['education']])

       education
0              0
1              0
2              0
3              0
4              1
...          ...
11157          2
11158          0
11159          0
11160          0
11161          0

[11162 rows x 1 columns]


In [ ]:
# Define the mapping dictionary
mapping_default = {
    'no': 0,
    'yes': 1
}

# Replace values in the "default" column using the mapping dictionary
data['default'] = data['default'].map(mapping_default)

# Display the result
print(data[['default']])

       default
0            0
1            0
2            0
3            0
4            0
...        ...
11157        0
11158        0
11159        0
11160        0
11161        0

[11162 rows x 1 columns]


In [ ]:
# Define the mapping dictionary
mapping_housing = {
    'no': 0,
    'yes': 1
}

# Replace values in the "housing" column using the mapping dictionary
data['housing'] = data['housing'].map(mapping_housing)

# Display the result
print(data[['housing']])

       housing
0            1
1            0
2            1
3            1
4            0
...        ...
11157        1
11158        0
11159        0
11160        0
11161        0

[11162 rows x 1 columns]


In [ ]:
# Define the mapping dictionary
mapping_loan = {
    'no': 0,
    'yes': 1
}

# Replace values in the "loan" column using the mapping dictionary
data['loan'] = data['loan'].map(mapping_loan)

# Display the result
print(data[['loan']])

       loan
0         0
1         0
2         0
3         0
4         0
...     ...
11157     0
11158     0
11159     0
11160     1
11161     0

[11162 rows x 1 columns]


In [ ]:
# Define the mapping dictionary
mapping_contact = {
    'cellular': 0,
    'unknown': 1,
    'telephone': 2
}

# Replace values in the "contact" column using the mapping dictionary
data['contact'] = data['contact'].map(mapping_contact)

# Display the result
print(data[['contact']])

       contact
0            1
1            1
2            1
3            1
4            1
...        ...
11157        0
11158        1
11159        0
11160        0
11161        0

[11162 rows x 1 columns]


In [ ]:
# Define the mapping dictionary
mapping_month = {
    'jan': 1,
    'feb': 2,
    'mar': 3,
    'apr': 4,
    'may': 5,
    'jun': 6,
    'jul': 7,
    'aug': 8,
    'sep': 9,
    'oct': 10,
    'nov': 11,
    'dec': 12
}

# Replace values in the "month" column using the mapping dictionary
data['month'] = data['month'].map(mapping_month)

# Display the result
print(data[['month']])

       month
0          5
1          5
2          5
3          5
4          5
...      ...
11157      4
11158      6
11159      8
11160      5
11161      7

[11162 rows x 1 columns]


In [ ]:
# Define the mapping dictionary
mapping_poutcome = {
    'unknown': 0,
    'failure': 1,
    'success': 2,
    'other': 3
}

# Replace values in the "poutcome" column using the mapping dictionary
data['poutcome'] = data['poutcome'].map(mapping_poutcome)

# Display the result
print(data[['poutcome']])

       poutcome
0             0
1             0
2             0
3             0
4             0
...         ...
11157         0
11158         0
11159         0
11160         1
11161         0

[11162 rows x 1 columns]


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        11162 non-null  int64 
 1   job        11162 non-null  int64 
 2   marital    11162 non-null  int64 
 3   education  11162 non-null  int64 
 4   default    11162 non-null  int64 
 5   balance    11162 non-null  int64 
 6   housing    11162 non-null  int64 
 7   loan       11162 non-null  int64 
 8   contact    11162 non-null  int64 
 9   day        11162 non-null  int64 
 10  month      11162 non-null  int64 
 11  duration   11162 non-null  int64 
 12  campaign   11162 non-null  int64 
 13  pdays      11162 non-null  int64 
 14  previous   11162 non-null  int64 
 15  poutcome   11162 non-null  int64 
 16  deposit    11162 non-null  object
dtypes: int64(16), object(1)
memory usage: 1.4+ MB


In [ ]:
# Seperating predictors and target variables
print(type(data))
X = data.iloc[:,0:16]
Y = list(data.iloc[:,-1])

print(X)
print('\n')
print(Y)

<class 'pandas.core.frame.DataFrame'>
       age  job  marital  education  default  balance  housing  loan  contact  \
0       59    3        0          0        0     2343        1     0        1   
1       56    3        0          0        0       45        0     0        1   
2       41    2        0          0        0     1270        1     0        1   
3       55    4        0          0        0     2476        1     0        1   
4       54    3        0          1        0      184        0     0        1   
...    ...  ...      ...        ...      ...      ...      ...   ...      ...   
11157   33    1        1          2        0        1        1     0        0   
11158   39    4        0          0        0      733        0     0        1   
11159   32    2        1          0        0       29        0     0        0   
11160   43    2        0          0        0        0        0     1        0   
11161   34    2        0          0        0        0        0     0   

In [ ]:
#Encoding the Y variable
#1-->yes and 0-->no
y_num =[]
for element in Y:
  if element == 'yes':
    y_num.append(1)
  else:
    y_num.append(0)

print(y_num)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
#preprocessing X variable
scaler = StandardScaler()
x_std = scaler.fit_transform(X) #transform x variable  in starndaized format
print (x_std[0])

[ 1.49150488  0.06659545 -0.78974281 -0.85672608 -0.12361659  0.25252514
  1.05527951 -0.38792323  1.07652967 -1.26574556 -0.46287434  1.93022589
 -0.55416834 -0.48118405 -0.36325984 -0.52312734]


In [ ]:
# Converting Y variable into a numpy array
y_num = np.array(y_num) #changes are reflected in tensor also.

In [ ]:
type(y_num)

numpy.ndarray

In [ ]:
type(x_std)

numpy.ndarray

In [ ]:
#Convert all into tensors
#Considering all as training data, not considering the evaluation
x_train =torch.tensor(x_std)
y_train =torch.tensor(y_num)

In [ ]:
print(y_train)

tensor([1, 1, 1,  ..., 0, 0, 0])


In [ ]:
print(x_train)

tensor([[ 1.4915,  0.0666, -0.7897,  ..., -0.4812, -0.3633, -0.5231],
        [ 1.2397,  0.0666, -0.7897,  ..., -0.4812, -0.3633, -0.5231],
        [-0.0195, -0.2971, -0.7897,  ..., -0.4812, -0.3633, -0.5231],
        ...,
        [-0.7750, -0.2971,  0.6544,  ..., -0.4812, -0.3633, -0.5231],
        [ 0.1484, -0.2971, -0.7897,  ...,  1.1096,  1.8183,  0.6492],
        [-0.6071, -0.2971, -0.7897,  ..., -0.4812, -0.3633, -0.5231]],
       dtype=torch.float64)


In [ ]:
print(y_train.shape)

#transforming y_train in to a 2d tensor
y_train = y_train.unsqueeze(1)
print(y_train)

torch.Size([11162])
tensor([[1],
        [1],
        [1],
        ...,
        [0],
        [0],
        [0]])


In [ ]:
print(y_train.shape)

torch.Size([11162, 1])


In [ ]:
print(x_train.shape)

torch.Size([11162, 16])


#Dataset Creation

In [ ]:
#Create and load mini batches
class BankMarketingDataset(Dataset):
  def __init__(self,x,y):
    self.x =x
    self.y = y

  def __getitem__(self,index):
    return self.x[index], self.y[index]

  def __len__(self):
    return len(self.y)

In [ ]:
#Create actual dataset from the class
Dataset =BankMarketingDataset(x_train,y_train)
len(Dataset)

11162

In [ ]:
#Iterable data components
train_loader=DataLoader(dataset=Dataset, batch_size=config.batch_size,shuffle=True)

In [ ]:
#demostrate the data iteration
for (x_batch,y_batch) in train_loader:
  print(f"X shape: {x_batch.shape}")
  print(f"Y shape: {y_batch.shape}")
  break

X shape: torch.Size([64, 16])
Y shape: torch.Size([64, 1])


#Define Neutral network architecture

In [ ]:
#neutral network
class Perceptron(nn.Module):
  def __init__(self,input_features):
    #inherit all the features from parent class
    super(Perceptron, self).__init__()
    self.fc1 = nn.Linear(input_features, 4)
    self.fc2 = nn.Linear(4,2)
    self.fc3 = nn.Linear(2,1)
    self.tanh = nn.Tanh()
    self.sigmoid = nn.Sigmoid()

  #method to build the network flow

  def forward(self, x):
    out = self.fc1(x)
    out = self.tanh(out)
    out = self.fc2(out)
    out = self.tanh(out)
    out = self.fc3(out)
    out = self.sigmoid(out)

    return out

#Training Process

In [ ]:
# Instantiate the network and the components
perceptron = Perceptron(16)

loss_function = torch.nn.BCELoss(reduction= 'mean')

optimizer = torch.optim.SGD(perceptron.parameters(), lr =config.lr , momentum=0.8)

In [ ]:
#Start the training process

for epoch in range(config.epochs):

  for(x_train, y_train) in train_loader:
    #network expect evey thing in float
    x_train = x_train.float()
    y_train = y_train.float()

    #forward propagation
    outputs = perceptron(x_train)

    #compare with actual and calculate loss
    loss = loss_function(outputs, y_train)

    #Clearing the gradient
    optimizer.zero_grad()

    #backpropagation
    loss.backward()

    #update weights
    optimizer.step()

    outputs = (outputs > 0.5).float()

    accuracy = (outputs == y_train).float().mean()

    #printing results into w&b
    metrics = {
        "Epoch": (epoch+1)/(config.epochs),
        "Loss" : loss,
        "Accuracy" : accuracy
    }

    wandb.log(metrics)

    print(f"Epoch: {(epoch+1)}/{(config.epochs)} , Loss: {loss}, Accuracy: {accuracy}")

wandb.finish()



Streaming output truncated to the last 5000 lines.
Epoch: 22/50 , Loss: 0.29460886120796204, Accuracy: 0.921875
Epoch: 22/50 , Loss: 0.49002060294151306, Accuracy: 0.75
Epoch: 22/50 , Loss: 0.44699394702911377, Accuracy: 0.84375
Epoch: 22/50 , Loss: 0.42243561148643494, Accuracy: 0.75
Epoch: 22/50 , Loss: 0.4204719364643097, Accuracy: 0.8125
Epoch: 22/50 , Loss: 0.5158679485321045, Accuracy: 0.75
Epoch: 22/50 , Loss: 0.38604360818862915, Accuracy: 0.828125
Epoch: 22/50 , Loss: 0.3502320349216461, Accuracy: 0.859375
Epoch: 22/50 , Loss: 0.36428022384643555, Accuracy: 0.84375
Epoch: 22/50 , Loss: 0.403886079788208, Accuracy: 0.84375
Epoch: 22/50 , Loss: 0.35268908739089966, Accuracy: 0.890625
Epoch: 22/50 , Loss: 0.40047842264175415, Accuracy: 0.828125
Epoch: 22/50 , Loss: 0.4794465899467468, Accuracy: 0.796875
Epoch: 22/50 , Loss: 0.3748459219932556, Accuracy: 0.828125
Epoch: 22/50 , Loss: 0.32446107268333435, Accuracy: 0.859375
Epoch: 22/50 , Loss: 0.33258023858070374, Accuracy: 0.8906

Accuracy,▁▂▂▅▂▆▆▆▆▆▅▄▄▆▄▆▆▅▄▆▂▅▆▅▅▃▂▆▅▄▄▆▄▆▄▆▄▅▄█
Epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Loss,█▆▇▄▇▃▃▄▃▄▄▆▅▅▄▄▄▄▅▄▇▃▃▄▄▅▆▄▅▄▅▄▅▅▆▃▄▄▄▁
Accuracy,0.88462
Epoch,1.0
Loss,0.27056
